In [10]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

# Path to the fingerprint dataset
dataset_path = "C:/Users/vrlit/Downloads/fingerprint/SOCOFing"

# Load and preprocess the dataset
X = []
y = []

# Load real fingerprints
real_path = os.path.join(dataset_path, "Real")
for file in os.listdir(real_path)[:1000]:
    image = cv2.imread(os.path.join(real_path, file), cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (128, 128))  # Resize to a consistent size
    X.append(image)
    y.append(1)  # Label 1 for real fingerprints

# Load altered fingerprints
altered_path = os.path.join(dataset_path, "Altered/Altered-Easy")
for file in os.listdir(altered_path)[:1000]:
    image = cv2.imread(os.path.join(altered_path, file), cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (128, 128))  # Resize to a consistent size
    X.append(image)
    y.append(0)  # Label 0 for altered fingerprints

# Convert the data to numpy arrays
X = np.array(X)
y = np.array(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the image data
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# Reshape the data for CNN input
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# Define the CNN model architecture
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(128, 128, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the CNN model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Make predictions using the CNN model
sample_path = "Downloads/fingerprint/SOCOFing/Altered/Altered-Easy/150__M_Right_index_finger_Obl.BMP"
sample_image = cv2.imread(sample_path, cv2.IMREAD_GRAYSCALE)
sample_image = cv2.resize(sample_image, (128, 128))
sample_image = np.expand_dims(sample_image, axis=0)
sample_image = np.expand_dims(sample_image, axis=-1)
sample_image = sample_image.astype("float32") / 255.0

cnn_prediction = model.predict(sample_image)

# Linear Regression Model
# Reshape the data for linear regression input
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Create the linear regression model
linear_model = LinearRegression()

# Train the linear regression model
linear_model.fit(X_train_flat, y_train)

# Make predictions using the linear regression model
y_pred = linear_model.predict(X_test_flat)
y_pred_binary = np.where(y_pred > 0.5, 1, 0)

# Calculate the accuracy of the linear regression model
accuracy = accuracy_score(y_test, y_pred_binary)
print("Linear Regression Accuracy:", accuracy)


Epoch 1/10
50/50 [==============================] - 31s 562ms/step - loss: 0.7113 - accuracy: 0.5419 - val_loss: 0.6929 - val_accuracy: 0.5025
Epoch 2/10
50/50 [==============================] - 26s 523ms/step - loss: 0.6931 - accuracy: 0.4969 - val_loss: 0.6923 - val_accuracy: 0.6100
Epoch 3/10
50/50 [==============================] - 27s 544ms/step - loss: 0.6928 - accuracy: 0.5119 - val_loss: 0.6814 - val_accuracy: 0.5200
Epoch 4/10
50/50 [==============================] - 27s 532ms/step - loss: 0.6966 - accuracy: 0.5575 - val_loss: 0.6821 - val_accuracy: 0.5450
Epoch 5/10
50/50 [==============================] - 26s 521ms/step - loss: 0.6806 - accuracy: 0.5650 - val_loss: 0.6926 - val_accuracy: 0.5000
Epoch 6/10
50/50 [==============================] - 26s 515ms/step - loss: 0.6813 - accuracy: 0.5763 - val_loss: 0.6746 - val_accuracy: 0.5925
Epoch 7/10
50/50 [==============================] - 26s 513ms/step - loss: 0.6461 - accuracy: 0.6156 - val_loss: 0.6403 - val_accuracy: 0.6175